In [1]:
import networkx as nx
import collections
import random
from heapq import heappush, heappop
from dataclasses import dataclass, field
from typing import Dict, Any
from bokeh.plotting import figure, show
from bokeh.models import GraphRenderer, StaticLayoutProvider, Scatter, LabelSet, ColumnDataSource
from bokeh.io import output_notebook

output_notebook()

BASE_FARE = 2.0
TIME_FACTOR = 0.15
DISTANCE_FACTOR = 1.0
PRIORITY_FACTOR = 0.2

@dataclass(order=True)
class Event:
    t: int
    action: str
    spaces: Dict[str, Any] = field(compare=False)

class Driver:
    def __init__(self, id, location, simulation):
        self.id = id
        self.location = location
        self.status = "idle"
        self.history = collections.deque(maxlen=10)
        self.current_rider = None
        self.ride_start_time = None
        self.sim = simulation


    def move(self, destination):
        self.location = destination
        self.history.append(self.sim.node_queue_lengths[destination])

    def accept_ride(self, rider):
        self.status = "occupied"
        self.current_rider = rider
        self.ride_start_time = self.sim.time
        print(f"Driver {self.id} accepted rider {rider.id} at time {self.sim.time}")

class Rider:
    def __init__(self, id, origin, destination):
        self.id = id
        self.origin = origin
        self.destination = destination

class Simulation:
    def __init__(self, graph):
        self.graph = graph
        self.drivers = []
        self.ride_queue = collections.deque()
        self.time = 0
        self.data = []
        self.ride_data = []
        self.event_queue = []

        self.node_queue_lengths = {node: 0 for node in self.graph.nodes()}

    def add_drivers(self, num_drivers):
        for i in range(num_drivers):
            start_node = random.choice(list(self.graph.nodes()))
            self.drivers.append(Driver(i, start_node, self))
            print(f"Added driver {i} at location {start_node}")  # Print the actual coordinates

    def generate_riders(self, event_time):
        origin = random.choice(list(self.graph.nodes()))
        destination = random.choice([node for node in self.graph.nodes() if node != origin])
        rider = Rider(event_time, origin, destination)
        self.ride_queue.append(rider)
        self.node_queue_lengths[origin] += 1
        print(f"Generated rider {rider.id} from {rider.origin} to {rider.destination} at time {event_time}")
        print(f"Current ride queue: {[r.id for r in self.ride_queue]}")  # Debugging line

    def calculate_dipr_priorities(self):
        alpha = 0.8
        priorities = {}
        for node in self.graph.nodes():
            priority = 0
            for driver in self.drivers:
                if node in driver.history:
                    priority += sum([alpha**(len(driver.history) - i - 1) * q
                                    for i, q in enumerate(driver.history) if q is not None])
            priorities[node] = priority
        return priorities

    def assign_drivers(self, priorities):
        idle_drivers = [d for d in self.drivers if d.status == "idle"]
        print(f"Idle drivers at time {self.time}: {[d.id for d in idle_drivers]}")
        for driver in idle_drivers:
            if self.ride_queue:
                rider = self.ride_queue.popleft()
                path = find_shortest_path(self.graph, driver.location, rider.origin, priorities)
                if path:
                    price = calculate_price(path, priorities, self.graph)
                    driver.accept_ride(rider)
                    self.node_queue_lengths[rider.origin] -= 1
                    driver.move(rider.origin)
                    ride_time = self.calculate_ride_time(rider.origin, rider.destination)
                    complete_ride_event = Event(t=self.time + ride_time, action="Complete Ride", spaces={"driver": driver, "rider": rider, "start_time": self.time})
                    heappush(self.event_queue, complete_ride_event)
                    print(f"Driver {driver.id} assigned to rider {rider.id}, ride completion scheduled at {self.time + ride_time}")

    def calculate_ride_time(self, origin, destination):
        path = nx.shortest_path(self.graph, origin, destination, weight='weight')
        return sum([self.graph[u][v]['weight'] for u, v in zip(path[:-1], path[1:])])

    def step(self):
        while self.event_queue and self.event_queue[0].t <= self.time:
            current_event = heappop(self.event_queue)
            self.process_event(current_event)
        
        self.time += 1
        
        priorities = self.calculate_dipr_priorities()
        self.assign_drivers(priorities)
        self.reposition_drivers()
        self.collect_data()

    def reposition_drivers(self):
        for driver in self.drivers:
            if driver.status == "idle":
                neighbors = list(self.graph.neighbors(driver.location))
                if neighbors:
                    driver.move(random.choice(neighbors))

    def collect_data(self):
        completed_rides = [ride for ride in self.ride_data if ride["time"] <= self.time]
        avg_wait_time = 0
        total_rides = 0
        avg_ride_time = 0
        avg_ride_price = 0

        data_point = {
            "time": self.time,
            "avg_wait_time": 0,
            "avg_ride_time": 0,
            "avg_ride_price": 0,
            "num_rides": 0,
            "idle_drivers": len([d for d in self.drivers if d.status == "idle"]),
            "queue_length": len(self.ride_queue)
        }

        if completed_rides:
            for ride in completed_rides:
                avg_wait_time += ride["wait_time"]
                avg_ride_time += ride["ride_time"]
                avg_ride_price += ride["price"]
                total_rides += 1
            avg_wait_time /= total_rides
            avg_ride_time /= total_rides
            avg_ride_price /= total_rides

            data_point["avg_wait_time"] = avg_wait_time
            data_point["avg_ride_time"] = avg_ride_time
            data_point["avg_ride_price"] = avg_ride_price
            data_point["num_rides"] = total_rides

        self.data.append(data_point)

    def process_event(self, event: Event):
        print(f"Processing event: {event}")
        if event.action == "Request Ride":
            self.generate_riders(event.t)
        elif event.action == "Complete Ride":
            driver = event.spaces["driver"]
            rider = event.spaces["rider"]
            start_time = event.spaces["start_time"]
            if driver and rider and start_time is not None:
                ride_time = event.t - start_time
                wait_time = start_time - rider.id
                self.ride_data.append({
                    "wait_time": wait_time,
                    "price": calculate_price(
                        nx.shortest_path(self.graph, rider.origin, rider.destination),
                        self.calculate_dipr_priorities(),
                        self.graph
                    ),
                    "ride_time": ride_time,
                    "driver_id": driver.id,
                    "rider_id": rider.id,
                    "time": event.t  # Store the completion timestamp
                })
                print(f"Ride completed by driver {driver.id} for rider {rider.id} with ride time {ride_time}")
                driver.status = "idle"
                driver.current_rider = None
                driver.ride_start_time = None
            print(f"Processing event: {event}")
            if event.action == "Request Ride":
                self.generate_riders(event.t)
            elif event.action == "Complete Ride":
                driver = event.spaces["driver"]
                rider = event.spaces["rider"]
                start_time = event.spaces["start_time"]
                if driver and rider and start_time is not None:
                    ride_time = event.t - start_time
                    wait_time = start_time - rider.id
                    self.ride_data.append({
                        "wait_time": wait_time,
                        "price": calculate_price(
                            nx.shortest_path(self.graph, rider.origin, rider.destination),
                            self.calculate_dipr_priorities(),
                            self.graph
                        ),
                        "ride_time": ride_time,
                        "driver_id": driver.id,
                        "rider_id": rider.id,
                        "time": self.time
                    })
                    print(f"Ride completed by driver {driver.id} for rider {rider.id} with ride time {ride_time}")
                    driver.status = "idle"
                    driver.current_rider = None
                    driver.ride_start_time = None

# Functions
def calculate_price(path, priorities, graph):
    distance = sum([graph[u][v]['weight'] for u, v in zip(path[:-1], path[1:])])
    time = distance / 10.0
    priority = priorities[path[-1]]
    return BASE_FARE + TIME_FACTOR * time + DISTANCE_FACTOR * distance + PRIORITY_FACTOR * priority

def find_shortest_path(graph, origin, destination, priorities):
    queue = [(0, origin, [])]
    visited = set()
    while queue:
        (cost, node, path) = heappop(queue)
        if node not in visited:
            visited.add(node)
            path = path + [node]
            if node == destination:
                return path
            for neighbor in graph.neighbors(node):
                if neighbor not in visited:
                    priority_cost = priorities.get(neighbor, 0)
                    new_cost = cost + graph[node][neighbor]['weight'] + priority_cost
                    heappush(queue, (new_cost, neighbor, path))
    return None

class SimulationVisualizer:
    def __init__(self, simulation):
        self.simulation = simulation

    def draw_graph(self):
        plot = figure(title=f"Time: {self.simulation.time}", x_range=(-1, 4), y_range=(-1, 4), tools="", toolbar_location=None)

        graph_renderer = GraphRenderer()

        # Define the graph layout with string keys
        graph_layout = {str(node): node for node in self.simulation.graph.nodes()}
        graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

        # Create nodes
        node_indices = [str(node) for node in self.simulation.graph.nodes()]
        graph_renderer.node_renderer.data_source.add(node_indices, 'index')
        graph_renderer.node_renderer.glyph = Scatter(size=15, fill_color='skyblue')

        # Create edges
        start_indices = [str(edge[0]) for edge in self.simulation.graph.edges()]
        end_indices = [str(edge[1]) for edge in self.simulation.graph.edges()]
        graph_renderer.edge_renderer.data_source.data = dict(start=start_indices, end=end_indices)

        plot.renderers.append(graph_renderer)

        # Add driver positions with string keys
        driver_data = {
            'x': [driver.location[0] for driver in self.simulation.drivers],
            'y': [driver.location[1] for driver in self.simulation.drivers],
            'driver': [f'D{driver.id}' for driver in self.simulation.drivers]
        }
        driver_source = ColumnDataSource(driver_data)
        plot.scatter('x', 'y', size=10, color='red', source=driver_source)
        driver_labels = LabelSet(x='x', y='y', text='driver', source=driver_source)
        plot.add_layout(driver_labels)

        # Add rider positions with string keys
        if self.simulation.ride_queue:
            rider_data = {
                'x': [rider.origin[0] for rider in self.simulation.ride_queue],
                'y': [rider.origin[1] for rider in self.simulation.ride_queue],
                'rider': [f'R{rider.id}' for rider in self.simulation.ride_queue]
            }
            rider_source = ColumnDataSource(rider_data)
            plot.scatter('x', 'y', size=20, color='blue', source=rider_source)
            rider_labels = LabelSet(x='x', y='y', text='rider', source=rider_source)
            plot.add_layout(rider_labels)

        # Add current ride positions (riders in cars) if any
        for driver in self.simulation.drivers:
            if driver.current_rider is not None:
                rider = driver.current_rider
                plot.scatter([driver.location[0]], [driver.location[1]], size=10, color='green')
                plot.text([driver.location[0]], [driver.location[1]], text=[f'R{rider.id}'], text_color='white')

        show(plot)

# Initialize the graph and simulation as before
graph = nx.DiGraph()

for i in range(4):
    for j in range(4):
        graph.add_node((i, j))

for i in range(4):
    for j in range(3):
        graph.add_edge((i, j), (i, j + 1), weight=2)
        graph.add_edge((i, j + 1), (i, j), weight=2)
        graph.add_edge((j, i), (j + 1, i), weight=2)
        graph.add_edge((j + 1, i), (j, i), weight=2)

# Start simulation
sim = Simulation(graph)
sim.add_drivers(10)
visualizer = SimulationVisualizer(sim)
visualizer.draw_graph()

# Generate initial ride request events
for t in [5, 10, 25]:
    request_ride_event = Event(t=t, action="Request Ride", spaces={})
    heappush(sim.event_queue, request_ride_event)

# Run the visualizer
visualizer = SimulationVisualizer(sim)
for _ in range(20):
    sim.step()
    visualizer.draw_graph()
    
    



Loading BokehJS ...

Added driver 0 at location (3, 2)
Added driver 1 at location (2, 2)
Added driver 2 at location (1, 2)
Added driver 3 at location (0, 3)
Added driver 4 at location (3, 2)
Added driver 5 at location (2, 3)
Added driver 6 at location (1, 0)
Added driver 7 at location (1, 2)
Added driver 8 at location (1, 0)
Added driver 9 at location (2, 3)


Idle drivers at time 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 3, 1), (1, 1, 2), (2, 1, 3), (3, 0, 2), (4, 3, 3), (5, 2, 2), (6, 2, 0), (7, 1, 1), (8, 0, 0), (9, 1, 3)]


Idle drivers at time 2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 3, 0), (1, 1, 1), (2, 2, 3), (3, 1, 2), (4, 3, 2), (5, 2, 1), (6, 3, 0), (7, 1, 2), (8, 1, 0), (9, 2, 3)]


Idle drivers at time 3: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 2, 0), (1, 1, 0), (2, 2, 2), (3, 1, 1), (4, 2, 2), (5, 2, 0), (6, 2, 0), (7, 1, 3), (8, 1, 1), (9, 3, 3)]


...

Processing event: Event(t=15, action='Complete Ride', spaces={'driver': <__main__.Driver object at 0x11899ffb0>, 'rider': <__main__.Rider object at 0x169511310>, 'start_time': 11})
Ride completed by driver 0 for rider 10 with ride time 4
Processing event: Event(t=15, action='Complete Ride', spaces={'driver': <__main__.Driver object at 0x11899ffb0>, 'rider': <__main__.Rider object at 0x169511310>, 'start_time': 11})
Ride completed by driver 0 for rider 10 with ride time 4
Idle drivers at time 16: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 0, 0), (1, 3, 1), (2, 1, 2), (3, 1, 2), (4, 0, 1), (5, 2, 3), (6, 2, 1), (7, 0, 1), (8, 1, 0), (9, 0, 3)]


Idle drivers at time 17: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 1, 0), (1, 2, 1), (2, 0, 2), (3, 1, 3), (4, 1, 1), (5, 2, 2), (6, 2, 0), (7, 0, 0), (8, 1, 1), (9, 1, 3)]


Idle drivers at time 18: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 2, 0), (1, 1, 1), (2, 0, 3), (3, 2, 3), (4, 1, 2), (5, 2, 3), (6, 1, 0), (7, 1, 0), (8, 1, 2), (9, 0, 3)]


Idle drivers at time 19: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 1, 0), (1, 0, 1), (2, 1, 3), (3, 1, 3), (4, 0, 2), (5, 2, 2), (6, 2, 0), (7, 0, 0), (8, 1, 1), (9, 1, 3)]


Idle drivers at time 20: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 0, 0), (1, 0, 0), (2, 1, 2), (3, 1, 2), (4, 0, 1), (5, 1, 2), (6, 2, 1), (7, 0, 1), (8, 2, 1), (9, 2, 3)]
